In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
net = keras.Sequential([
    # conv 1
    layers.Conv2D(96, kernel_size=11, strides=4, activation='relu'), 
    layers.MaxPool2D(pool_size=3, strides=2), 
    # conv 2
    layers.Conv2D(256, kernel_size=5, padding='same', activation='relu'), 
    layers.MaxPool2D(pool_size=3, strides=2), 
    # conv 3 4 5 
    layers.Conv2D(384, kernel_size=3, padding='same', activation='relu'), 
    layers.Conv2D(384, kernel_size=3, padding='same', activation='relu'), 
    layers.Conv2D(256, kernel_size=3, padding='same', activation='relu'), 
    layers.MaxPool2D(pool_size=3, strides=2), 
    # dense 
    layers.Flatten(), 
    layers.Dense(4096, activation='relu'), 
    layers.Dropout(0.5), 
    layers.Dense(4096, activation='relu'), 
    layers.Dropout(0.5), 
    layers.Dense(10, activation='sigmoid'),
])

In [6]:
# print(net.summary())

X = tf.random.uniform((1,224,224,1))
for layer in net.layers:
    X = layer(X)
    print(layer.name, 'output shape\t', X.shape)


conv2d output shape	 (1, 54, 54, 96)
max_pooling2d output shape	 (1, 26, 26, 96)
conv2d_1 output shape	 (1, 26, 26, 256)
max_pooling2d_1 output shape	 (1, 12, 12, 256)
conv2d_2 output shape	 (1, 12, 12, 384)
conv2d_3 output shape	 (1, 12, 12, 384)
conv2d_4 output shape	 (1, 12, 12, 256)
max_pooling2d_2 output shape	 (1, 5, 5, 256)
flatten output shape	 (1, 6400)
dense output shape	 (1, 4096)
dropout output shape	 (1, 4096)
dense_1 output shape	 (1, 4096)
dropout_1 output shape	 (1, 4096)
dense_2 output shape	 (1, 10)


In [7]:
class DataLoader():
    def __init__(self):
        fashion_mnist = tf.keras.datasets.fashion_mnist
        (self.X_train, self.y_train), (self.X_test, self.y_test) = fashion_mnist.load_data()
        self.X_train = np.expand_dims(self.X_train.astype(np.float32)/255.0,axis=-1)
        self.X_test = np.expand_dims(self.X_test.astype(np.float32)/255.0,axis=-1)
        self.y_train = self.y_train.astype(np.int32)
        self.y_test = self.y_test.astype(np.int32)
        self.num_train, self.num_test = self.X_train.shape[0], self.X_test.shape[0]

    def get_batch_train(self, batch_size):
        index = np.random.randint(0, self.num_train, batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.X_train[index],224,224,)
        return resized_images.numpy(), self.y_train[index]

    def get_batch_test(self, batch_size):
        index = np.random.randint(0, self.num_test, batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.X_test[index],224,224,)
        return resized_images.numpy(), self.y_test[index]

batch_size = 128
dataLoader = DataLoader()
x_batch, y_batch = dataLoader.get_batch_train(batch_size)
print("x_batch shape:",x_batch.shape,"y_batch shape:", y_batch.shape)


x_batch shape: (128, 224, 224, 1) y_batch shape: (128,)


In [ ]:
net.compile(
    optimizer=keras.optimizers.SGD(
        learning_rate=0.01, momentum=0.0, nesterov=False
    ), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

epochs = 10
num_iter = dataLoader.num_train // batch_size

for ep in range(epochs):
    for n in range(num_iter):
        X_batch, y_batch = dataLoader.get_batch_train(batch_size)
        net.fit(X_batch, y_batch)
#         if n % 20 == 0:
#             net.save_weights("alexnet_weights.h5")


In [10]:
net.load_weights("alexnet_weights.h5")
X_test, y_test = dataLoader.get_batch_test(2000)
net.evaluate(X_test, y_test, verbose=2)

2000/2000 - 2s - loss: 0.2627 - accuracy: 0.9165


[0.26265823233127594, 0.9165]